# Phase 1: Planning & Model Setup

Make sure all requirements are installed before running any code!

### Step 1: Load & Benchmark Model in Plaintext

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import time

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

model.eval()

prompt = "Explain the benefits of homomorphic encryption."
inputs = tokenizer(prompt, return_tensors="pt")

start = time.time()
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=32)
end = time.time()

print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(f"Latency: {end - start:.2f} seconds")



/Users/rojo/Documents/repos/ZK-LLM-Turbo/split-inference-env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Explain the benefits of homomorphic encryption.
Latency: 0.58 seconds


### Step 2: Identify Linear vs Non-Linear Components

In [2]:
print(f"Latency: {end - start:.2f} seconds")
for name, module in model.named_modules():
    print(name, type(module))

Latency: 0.58 seconds
 <class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>
model <class 'transformers.models.llama.modeling_llama.LlamaModel'>
model.embed_tokens <class 'torch.nn.modules.sparse.Embedding'>
model.layers <class 'torch.nn.modules.container.ModuleList'>
model.layers.0 <class 'transformers.models.llama.modeling_llama.LlamaDecoderLayer'>
model.layers.0.self_attn <class 'transformers.models.llama.modeling_llama.LlamaAttention'>
model.layers.0.self_attn.q_proj <class 'torch.nn.modules.linear.Linear'>
model.layers.0.self_attn.k_proj <class 'torch.nn.modules.linear.Linear'>
model.layers.0.self_attn.v_proj <class 'torch.nn.modules.linear.Linear'>
model.layers.0.self_attn.o_proj <class 'torch.nn.modules.linear.Linear'>
model.layers.0.mlp <class 'transformers.models.llama.modeling_llama.LlamaMLP'>
model.layers.0.mlp.gate_proj <class 'torch.nn.modules.linear.Linear'>
model.layers.0.mlp.up_proj <class 'torch.nn.modules.linear.Linear'>
model.layers.0.mlp.down_proj <cla

### Step 3: Collect Baseline Accuracy

Run a small evaluation to get perplexity (language-model quality)

In [3]:
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import math

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
model.eval()

dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test[:1%]")  # small subset
encodings = tokenizer("\n\n".join(dataset["text"]), return_tensors="pt")

max_length = model.config.max_position_embeddings
stride = 512
nlls = []
for i in range(0, encodings.input_ids.size(1), stride):
    begin_loc = max(i + stride - max_length, 0)
    end_loc = i + stride
    input_ids = encodings.input_ids[:, begin_loc:end_loc]
    target_ids = input_ids.clone()
    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
        neg_log_likelihood = outputs.loss
    nlls.append(neg_log_likelihood)
ppl = math.exp(torch.stack(nlls).mean())
print(f"Baseline perplexity: {ppl:.2f}")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Generating validation split: 100%|██████████| 3760/3760 [00:00<00:00, 1400460.26 examples/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (2261 > 2048). Running this sequence through the model will result in indexing errors


Baseline perplexity: 6.13


### Step 4: Measure Memory Footprint

Use psutil or torch.cuda.memory_allocated() (if you had GPU).
On macOS CPU:

In [4]:
import psutil
process = psutil.Process()
mem_MB = process.memory_info().rss / 1024 ** 2
print(f"Memory used: {mem_MB:.1f} MB")

Memory used: 5953.2 MB


### Step 5: Log the Architecture (Linear vs Non-Linear)

Run this once and save to model_layers.txt:

In [5]:
with open("model_layers.txt", "w") as f:
    for name, module in model.named_modules():
        f.write(f"{name} : {type(module)}\n")
